In [1]:
# import libraries
import pandas as pd
# !pip install seaborn
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
# !pip install plotly
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
# from pandas_profiling import ProfileReport
from pandas.plotting import scatter_matrix
#!pip install pm4py
import pm4py
from pm4py.objects.log.util.log import project_traces
from pm4py.objects.log.util import interval_lifecycle
# !pip install zipfile
import zipfile

#!pip install warnings
import warnings
warnings.filterwarnings("ignore")

In [10]:
# load data
data = pd.read_csv('data/bpi_challenge_offer.csv', sep=',')
# data.head()
trace = pm4py.format_dataframe(data, case_id='Case ID', activity_key='Activity', timestamp_key="Start Timestamp")

In [11]:
# To extract target varaible, 
# if event starts with the name decleration rejected it is considered as rejected application
trace_log = pm4py.convert_to_event_log(trace)
accepted = []
for trace in trace_log:
    flag = False
    for i,event in enumerate(trace):
        if "O_Accepted" in event['concept:name']:
            flag = True
            break
    
    if flag:
        accepted.append(1)
    else:
        accepted.append(0)

# convert logs to dataframe
df = pm4py.convert_to_dataframe(trace_log)

In [12]:
# categoric data
# keeping only low cardinality values
cat_columns = df.select_dtypes(include='object')
cat_columns = cat_columns[['Accepted', 'Selected', 'LoanGoal', 'ApplicationType']]
cat_columns.head()

# numeric data
num_columns = df.select_dtypes(include='float64')
num_columns = num_columns[['RequestedAmount', 'FirstWithdrawalAmount', 'NumberOfTerms', 'MonthlyCost', 'CreditScore', 'OfferedAmount']]
num_columns.head()
cat_columns.shape, num_columns.shape

((147724, 4), (147724, 6))

In [13]:
filtered_rows = df[df['EventOrigin'] == 'Offer']
num_columns = filtered_rows.select_dtypes(include='number')
num_columns.columns
len(filtered_rows)

58989

# Random Forest Implementation

In [14]:
# concatenate feature columns and target column
data = pd.concat([cat_columns["Selected"], 
                             num_columns[["RequestedAmount", "OfferedAmount", 
                                          "FirstWithdrawalAmount", "MonthlyCost", "NumberOfTerms"]]], axis=1)
# drop rows with missing values
data = data.dropna()
# print(data)

In [15]:
# create target variable for prediction
import numpy as np
y = data['Selected']
y = np.array(y).astype(int) # make sure it is an np.array and not a series
x = data.drop(["Selected"], axis=1)

In [16]:
# Implementation of Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)

# fit model with 5-fold Cross Validation
from sklearn.model_selection import cross_validate
result = cross_validate(clf, x, y)  # (defaults) is 5-fold CV

In [17]:
# print evaluation metrics
print('fit_time:', result['fit_time'])
print('score_time:', result['score_time'])
print('test_score:', result['test_score'])
print('mean accuracy:', sum(result['test_score']) / len(result['test_score']))

fit_time: [9.83097148 9.6426537  9.68240952 9.62197304 9.35981941]
score_time: [0.30424976 0.29415226 0.29621053 0.30276012 0.27961397]
test_score: [0.59454913 0.61215792 0.60998317 0.61166573 0.59831744]
mean accuracy: 0.6053346784735049
